In [2]:
%pwd

'/net/tscratch/people/plgkingak'

In [3]:
path = "weights2weights/weights_datasets"
device = "cuda"

In [4]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets

In [4]:
weight_dimensions = torch.load(f"{path}/weight_dimensions.pt")

In [6]:
dim_df = pd.DataFrame(weight_dimensions).T

In [7]:
dim_df

,0,1
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.weight,"(320,)","(1, 320)"
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_B.weight,"(320,)","(320, 1)"
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.lora_A.weight,"(320,)","(1, 320)"
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.lora_B.weight,"(320,)","(320, 1)"
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_q.lora_A.weight,"(320,)","(1, 320)"
...,...,...
base_model.model.up_blocks.3.attentions.2.transformer_blocks.0.attn1.to_v.lora_B.weight,"(320,)","(320, 1)"
base_model.model.up_blocks.3.attentions.2.transformer_blocks.0.attn2.to_q.lora_A.weight,"(320,)","(1, 320)"
base_model.model.up_blocks.3.attentions.2.transformer_blocks.0.attn2.to_q.lora_B.weight,"(320,)","(320, 1)"
base_model.model.up_blocks.3.attentions.2.transformer_blocks.0.attn2.to_v.lora_A.weight,"(768,)","(1, 768)"


In [14]:
dim_df["use"] = 0
dim_df.loc[dim_df.index.str.contains("mid_block"), "use"] = 1

In [15]:
dim_df.use.sum()

8

In [22]:
dim_df["lengths"] = [a.numel() for a,_ in list(weight_dimensions.values())]

In [25]:
dim_df["start"] = dim_df.lengths.cumsum().shift(fill_value=0)
dim_df["end"] = dim_df.start + dim_df.lengths

In [28]:
dim_df[dim_df.use == 1]

,0,1,use,lengths,start,end
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.weight,"(1280,)","(1, 1280)",1,1280,35968,37248
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn1.to_q.lora_B.weight,"(1280,)","(1280, 1)",1,1280,37248,38528
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn1.to_v.lora_A.weight,"(1280,)","(1, 1280)",1,1280,38528,39808
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn1.to_v.lora_B.weight,"(1280,)","(1280, 1)",1,1280,39808,41088
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn2.to_q.lora_A.weight,"(1280,)","(1, 1280)",1,1280,41088,42368
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn2.to_q.lora_B.weight,"(1280,)","(1280, 1)",1,1280,42368,43648
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn2.to_v.lora_A.weight,"(768,)","(1, 768)",1,768,43648,44416
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn2.to_v.lora_B.weight,"(1280,)","(1280, 1)",1,1280,44416,45696


In [35]:
indices = np.concatenate([np.arange(row.start, row.end) for _, row in dim_df[dim_df.use == 1].iterrows()])
len(indices)


9728

In [5]:
weights = torch.load(f"{path}/identities/all_weights.pt", torch.device(device))

In [ ]:
torch.save(weights[:,indices], f'{path}/mid_block.pt')

In [4]:
dataset = torch.load(f'{path}/mid_block.pt', torch.device(device))

In [5]:
Dataset.from_dict({"data":dataset}).save_to_disk(f'{path}/mid_block')

Saving the dataset (0/6 shards):   0%|          | 0/64974 [00:00<?, ? examples/s]

In [8]:
d = Dataset.from_dict({"data":dataset})

In [9]:
d

Dataset({
    features: ['data'],
    num_rows: 64974
})

In [15]:
len(d[0]["data"])

9728

In [6]:
!nvidia-smi

Wed Jan  8 17:11:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:8D:00.0 Off |                    0 |
| N/A   27C    P0             73W /  400W |   25121MiB /  40960MiB |     39%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [9]:
import psutil

for i, chunk in enumerate(weights.split(10000)):
    dc = Dataset.from_dict({"data":chunk})
    dc.save_to_disk(f'{path}/full/{i}')
    print('RAM memory % used:', psutil.virtual_memory()[2])
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

Saving the dataset (0/8 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

RAM memory % used: 18.3
RAM Used (GB): 169.273266176


Saving the dataset (0/8 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

RAM memory % used: 18.3
RAM Used (GB): 168.787308544


Saving the dataset (0/8 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

...

RAM memory % used: 18.3
RAM Used (GB): 169.329750016


Saving the dataset (0/8 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

RAM memory % used: 18.4
RAM Used (GB): 169.76019456


Saving the dataset (0/4 shards):   0%|          | 0/4974 [00:00<?, ? examples/s]

RAM memory % used: 18.1
RAM Used (GB): 167.055147008
